In [67]:
import numpy as np
import pandas as pd
import random
import re
from collections import defaultdict

In [2]:
chunks=pd.read_csv("hash.txt",header=None,chunksize=1000000)

In [112]:
class HashLogLog:
    def __init__(self):
        random.seed(42)
        self.n_digits = 32
        self.m = 2 ** self.bits
        self.p = 4999999937 # 10 ** 9 + 9
        self.a = random.randint(1, self.m)
        self.b = random.randint(0, self.m)
    
    def get_hash(self, x):
        x = int(x, 16)
        bin_hash = bin(((self.a * x + self.b) % self.p) % self.m)[2:]
        return bin_hash.zfill(self.bits)

In [114]:
my_hash = HashLogLog()
bucket_max = [0]*64
with open('hash.txt') as f:
    for line in f:
        hashed = my_hash.get_hash(line)
        root = int(hashed[:6], 2)
        try:
            temp = hashed[6:].index('1')
        except:
            temp = len(hashed[6:])
        if temp > bucket_max[root]:
            bucket_max[root] = temp
        

In [125]:
estimate = int(2 ** (sum(bucket_max) / len(bucket_max)))
estimate

2052214

In [7]:
dict_of_buckets={}
for i in range (64):
    z=str(bin(i)[2:].zfill(6))
    dict_of_buckets["bucket"+z]=0
dict_of_buckets

{'bucket000000': 0,
 'bucket000001': 0,
 'bucket000010': 0,
 'bucket000011': 0,
 'bucket000100': 0,
 'bucket000101': 0,
 'bucket000110': 0,
 'bucket000111': 0,
 'bucket001000': 0,
 'bucket001001': 0,
 'bucket001010': 0,
 'bucket001011': 0,
 'bucket001100': 0,
 'bucket001101': 0,
 'bucket001110': 0,
 'bucket001111': 0,
 'bucket010000': 0,
 'bucket010001': 0,
 'bucket010010': 0,
 'bucket010011': 0,
 'bucket010100': 0,
 'bucket010101': 0,
 'bucket010110': 0,
 'bucket010111': 0,
 'bucket011000': 0,
 'bucket011001': 0,
 'bucket011010': 0,
 'bucket011011': 0,
 'bucket011100': 0,
 'bucket011101': 0,
 'bucket011110': 0,
 'bucket011111': 0,
 'bucket100000': 0,
 'bucket100001': 0,
 'bucket100010': 0,
 'bucket100011': 0,
 'bucket100100': 0,
 'bucket100101': 0,
 'bucket100110': 0,
 'bucket100111': 0,
 'bucket101000': 0,
 'bucket101001': 0,
 'bucket101010': 0,
 'bucket101011': 0,
 'bucket101100': 0,
 'bucket101101': 0,
 'bucket101110': 0,
 'bucket101111': 0,
 'bucket110000': 0,
 'bucket110001': 0,


In [8]:
dict_of_buckets={}
for i in range (64):
    z=str(bin(i)[2:].zfill(6))
    dict_of_buckets["bucket"+z]=0

my_hash = HashLogLog()

In [9]:
for chunk_n, chunk in enumerate(chunks):
    binary = chunk[0].apply(my_hash.leading_zeroes)
    for i in binary:
        root=str(i[0])
        current_max=int(dict_of_buckets["bucket"+root])
        if int(i[1]) > current_max:
            dict_of_buckets["bucket"+root]=int(i[1])
    print("Chunk number {}".format(chunk_n))

Chunk number 0
Chunk number 1
Chunk number 2
Chunk number 3
Chunk number 4
Chunk number 5
Chunk number 6
Chunk number 7
Chunk number 8
Chunk number 9
Chunk number 10
Chunk number 11
Chunk number 12
Chunk number 13
Chunk number 14
Chunk number 15
Chunk number 16
Chunk number 17
Chunk number 18
Chunk number 19
Chunk number 20
Chunk number 21
Chunk number 22
Chunk number 23
Chunk number 24
Chunk number 25
Chunk number 26
Chunk number 27
Chunk number 28
Chunk number 29
Chunk number 30
Chunk number 31
Chunk number 32
Chunk number 33
Chunk number 34
Chunk number 35
Chunk number 36
Chunk number 37
Chunk number 38
Chunk number 39
Chunk number 40
Chunk number 41
Chunk number 42
Chunk number 43
Chunk number 44
Chunk number 45
Chunk number 46
Chunk number 47
Chunk number 48
Chunk number 49
Chunk number 50
Chunk number 51
Chunk number 52
Chunk number 53
Chunk number 54
Chunk number 55
Chunk number 56
Chunk number 57
Chunk number 58
Chunk number 59
Chunk number 60
Chunk number 61
Chunk number 62
Ch

In [11]:
dict_of_buckets

{'bucket000000': 16,
 'bucket000001': 16,
 'bucket000010': 15,
 'bucket000011': 17,
 'bucket000100': 15,
 'bucket000101': 18,
 'bucket000110': 16,
 'bucket000111': 15,
 'bucket001000': 23,
 'bucket001001': 19,
 'bucket001010': 21,
 'bucket001011': 17,
 'bucket001100': 17,
 'bucket001101': 20,
 'bucket001110': 14,
 'bucket001111': 18,
 'bucket010000': 18,
 'bucket010001': 16,
 'bucket010010': 16,
 'bucket010011': 19,
 'bucket010100': 22,
 'bucket010101': 18,
 'bucket010110': 16,
 'bucket010111': 18,
 'bucket011000': 16,
 'bucket011001': 18,
 'bucket011010': 17,
 'bucket011011': 17,
 'bucket011100': 19,
 'bucket011101': 17,
 'bucket011110': 16,
 'bucket011111': 16,
 'bucket100000': 22,
 'bucket100001': 24,
 'bucket100010': 23,
 'bucket100011': 21,
 'bucket100100': 21,
 'bucket100101': 23,
 'bucket100110': 25,
 'bucket100111': 21,
 'bucket101000': 21,
 'bucket101001': 21,
 'bucket101010': 20,
 'bucket101011': 22,
 'bucket101100': 26,
 'bucket101101': 22,
 'bucket101110': 24,
 'bucket10111

In [12]:
list_of_values=list()
for i in dict_of_buckets:
    list_of_values.append(dict_of_buckets[i])
n=np.mean(list_of_values)
n

19.71875

In [13]:
estimated_cardinality=2**n
estimated_cardinality

862849.848130386